# Scrape wikipedia page.

Use the Notebook to build the code to scrape the following Wikipedia page,
- https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

### Imports

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

### Scrape table using pandas

In [29]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url).content
codes=pd.read_html(results)[0]

### Clean Not assigned boroughs.

In [30]:
p_codes=codes[codes['Borough']!='Not assigned']
p_codes.shape
p_codes.head()

### Remove duplicate Postcode

In [33]:
strJoin = lambda x:",".join(x.astype(str))  
asIs = lambda x: x  
temp1=p_codes.groupby('Postcode')['Neighbourhood'].agg([('Neighbourhood', ', '.join)]).reset_index()

temp2=p_codes.groupby('Postcode').first().reset_index()[['Postcode','Borough']]
#temp2=temp

temp3=temp2.join(temp1.set_index('Postcode'), on='Postcode')

p_codes=temp3
p_codes.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Check if Not assigned neighbourhood exist

In [34]:
test=p_codes[p_codes['Neighbourhood']=='Not assigned']
print(test.count())

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64


In [35]:
p_codes.shape

(103, 3)

In [36]:
#!pip install geocoder
#import geocoder # import geocoder

In [37]:
# loop until you get the coordinates
#column_names=['Postcode','Lattitude','Longitude']
#latlong=pd.DataFrame(columns=column_names);
#for postal_code in p_codes['Postcode']:
#    lat_lng_coords = None
#    while(lat_lng_coords is None):
#      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#      lat_lng_coords = g.latlng
#    print(lat_lng_coords)
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]
#    latlong.append({'Postcode':postal_code,'Lattitude':latitude,'Longitude':longitude},ignore_index=True)

# Get lattitude and longitude coodinates.

In [38]:
csv_path='http://cocl.us/Geospatial_data'
lat_long=pd.read_csv(csv_path)
lat_long.rename(columns={'Postal Code':'Postcode'},inplace=True)
lat_long.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge lattitude and longitude to neighbourhood dataframe.

In [39]:
merged=p_codes.join(lat_long.set_index('Postcode'),on='Postcode')
merged.reset_index()
merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
